In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# import data
df = pd.read_csv('deliverable_2.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420420 entries, 0 to 420419
Data columns (total 8 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Period                             420420 non-null  object 
 1   Primary classification             420420 non-null  object 
 2   Secondary classification variable  420420 non-null  object 
 3   Secondary classification value     352170 non-null  object 
 4   Variable                           420420 non-null  object 
 5   Statistic                          420420 non-null  object 
 6   Rounded N                          420420 non-null  float64
 7   Value                              309024 non-null  float64
dtypes: float64(2), object(6)
memory usage: 25.7+ MB


In [25]:
# Check value that do not has "hh" in the "Variable" column
df1 = df[['Variable','Value']][~df['Variable'].str.contains('hh')]
df1['Variable'].value_counts()

Variable
weighted_mean_Clean_elec_imp_kWh_d_mean           5390
weighted_mean_gas_fixed_expenditure_pc_total      5390
weighted_mean_total_expenditure                   5390
weighted_mean_gas_expenditure                     5390
weighted_mean_elec_expenditure                    5390
weighted_mean_total_variable_expenditure          5390
weighted_mean_gas_variable_expenditure            5390
weighted_mean_elec_variable_expenditure           5390
weighted_mean_total_fixed_expenditure_pc_total    5390
weighted_mean_Clean_gas_kWh_d_mean                5390
weighted_mean_total_fixed_expenditure             5390
weighted_mean_gas_fixed_expenditure               5390
weighted_mean_elec_fixed_expenditure_pc_total     5390
weighted_mean_elec_fixed_expenditure              5390
weighted_mean_gas_unit_cost                       5390
weighted_mean_elec_unit_cost                      5390
weighted_mean_gas_standing_charge                 5390
weighted_mean_elec_standing_charge                5390
w

In [26]:
df1.head()

,Variable,Value
0,weighted_mean_Clean_elec_imp_kWh_d_mean,8.990031
1,weighted_mean_Clean_elec_imp_kWh_d_mean,0.354596
2,weighted_mean_Clean_elec_imp_kWh_d_mean,2.946595
3,weighted_mean_Clean_elec_imp_kWh_d_mean,4.644354
4,weighted_mean_Clean_elec_imp_kWh_d_mean,6.992137


In [28]:
df1['Value'].loc[df1['Variable'] == 'weighted_mean_gas_variable_expenditure'].describe()

count    3999.000000
mean       69.811894
std        60.941536
min         0.391782
25%        23.515757
50%        58.025964
75%        98.565115
max       402.577087
Name: Value, dtype: float64

In [30]:
df1['Value'].loc[df1['Variable'] == 'weighted_mean_gas_unit_cost'].describe()

count    4050.000000
mean        0.059315
std         0.033443
min         0.000071
25%         0.038433
50%         0.070024
75%         0.086614
max         0.110956
Name: Value, dtype: float64

In [31]:
df1['Value'].loc[df1['Variable'] == 'weighted_mean_total_fixed_expenditure_pc_total'].describe()

count    3871.000000
mean       13.854947
std         9.396972
min         0.099875
25%         7.359553
50%        13.566034
75%        19.369901
max        48.111933
Name: Value, dtype: float64

In [32]:
df1['Value'].loc[df1['Variable'] == 'weighted_mean_gas_fixed_expenditure_pc_total'].describe()

count    3989.000000
mean       16.732590
std        13.729007
min         0.154820
25%         6.241931
50%        13.749397
75%        24.321675
max        75.817935
Name: Value, dtype: float64

In [34]:
df1['Value'].loc[df1['Variable'] == 'weighted_mean_elec_fixed_expenditure_pc_total'].describe()

count    4085.000000
mean       15.589251
std        10.638600
min         0.085530
25%         7.679316
50%        15.224513
75%        21.928377
max        53.822922
Name: Value, dtype: float64

In [33]:
df1['Value'].loc[df1['Variable'] == 'pc_disposable_income_spent_on_fuel_bills'].describe()

count    3751.000000
mean        9.300073
std         8.925573
min         0.040984
25%         3.071074
50%         7.004396
75%        13.009403
max        70.449816
Name: Value, dtype: float64

In [35]:
df['Statistic'].value_counts()

Statistic
mean           60060
sem            60060
quantile_5     60060
quantile_25    60060
quantile_50    60060
quantile_75    60060
quantile_95    60060
Name: count, dtype: int64